In [1]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
sys.path.append('../LIB/')
from env import ENV
from sklearn.preprocessing import normalize
from tqdm import tqdm

import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import pickle
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.utils import shuffle
import gc

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)

In [2]:
X_Train = pd.read_pickle(ENV.application_train_cleaned.value)
print('Train shape: {}'.format(X_Train.shape))

X_Test = pd.read_pickle(ENV.application_test_cleaned.value)
print('Test shape: {}'.format(X_Test.shape))

X_pos = pd.read_pickle(ENV.POS_CASH_balance_clean.value)
print('Installment shape: {}'.format(X_pos.shape))

X_pos_fe = pd.read_pickle(ENV.POS_CASH_balance_clean_rnn.value)
print('Installment shape: {}'.format(X_pos_fe.shape))

Train shape: (307511, 122)
Test shape: (48744, 121)
Installment shape: (10001358, 8)
Installment shape: (936325, 42)


In [3]:
def get_embeddings_index(sorted_df,nor_ebd):
    embeddings_index={}
    words_values = sorted_df['words'].values
    for index in range(len(words_values)):
        embeddings_index  [words_values[index]] = nor_ebd[index,:]
    return embeddings_index

def create_document(sorted_df):
    #Create document
    ids = sorted_df.SK_ID_CURR.values
    words = sorted_df.words.values
    document_dicts = {}

    id_list = []
    document_list = []

    for index in range(len(ids)) :
        if document_dicts.get(ids[index]) is None:
            document_dicts[ids[index]] = []
        document_dicts[ids[index]].append(words[index])

    for key in document_dicts :
        document_dicts[key] = ' '.join(document_dicts[key])
        id_list.append(key)
        document_list.append(document_dicts[key])


    df_doc = pd.DataFrame({'SK_ID_CURR':id_list, 'text':document_list})  
    df_doc_mapping  = df_doc.set_index('SK_ID_CURR').text

    train = X_Train[['SK_ID_CURR','TARGET']].copy()
    test = X_Test[['SK_ID_CURR']].copy()
    train['text'] = train.SK_ID_CURR.map(df_doc_mapping).fillna('notfound')
    test['text'] = test.SK_ID_CURR.map(df_doc_mapping).fillna('notfound')
    return train,test


def get_train_ebdMat(train,test,embeddings_index):
    X_train = train["text"].str.lower()
    X_test = test["text"].str.lower()
    y_train = train["TARGET"].values
    tok=text.Tokenizer(num_words=max_features,lower=True,filters='')
    tok.fit_on_texts(list(X_train)+list(X_test))
    X_train=tok.texts_to_sequences(X_train)
    X_test=tok.texts_to_sequences(X_test)
    x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
    x_test=sequence.pad_sequences(X_test,maxlen=maxlen)
    print('...get ebd mat')
    word_index = tok.word_index
    #prepare embedding matrix
    num_words = min(max_features, len(word_index) + 1)
    print('num of words: {}'.format(num_words))
    embedding_matrix = np.zeros((num_words, embed_size))
    print(embedding_matrix.shape)
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return x_train,x_test,y_train,embedding_matrix,num_words

class_ratio =  sum(X_Train.TARGET ==0)/sum(X_Train.TARGET ==1)
class_ratio =  1
def get_rnn_model(num_words,embed_size,embedding_matrix):
    sequence_input = Input(shape=(maxlen, ))
    
    x = Embedding(num_words, embed_size, weights=[embedding_matrix],trainable = False)(sequence_input)
    x = BatchNormalization()(x)
    x = SpatialDropout1D(0.2)(x)
#     x = Bidirectional(GRU(16, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
    x = Bidirectional(CuDNNGRU(8, return_sequences=True))(x)
    x = Conv1D(32, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
    x = BatchNormalization()(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool]) 
#     x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(1, activation="sigmoid")(x)
    model = Model(sequence_input, preds)
#     Adam,RMSprop,Adagrad,Adadelta,Adamax,Nadam
#     model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-4),metrics=['accuracy'])
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    return model

def train_each_epoch(x,y,batch_size,model):
    x,y = shuffle(x,y)
    model.fit(x, y, 
              batch_size=batch_size, 
              epochs=1,
              verbose=1,
              class_weight={0:1,1:class_ratio})
    return model

def load_model(model,filepath):
    model.load_weights(filepath)
    return model

def save_model(model, filepath):
    model.save_weights(filepath)

def train_each_fold(x,y,x_val,y_val,model,filepath,reportpath,predspath,
                    batch_size=512,total_epoch=40,patience=5,saving=True):
    ROC_AUC_SCORE = []
    for epoch in range(total_epoch):  
        model = train_each_epoch(x,y,batch_size,model)
        y_pred = model.predict(x_val,batch_size=3000,verbose=1)
        score = roc_auc_score(y_val,y_pred)
        if len(ROC_AUC_SCORE) == 0:
            if saving:
                save_model(model,filepath)
            best_score = 0 
            if saving:
                print('saving preds...')
                pickle.dump(y_pred,open(predspath,'wb'))
        else:
            best_score = max(ROC_AUC_SCORE)
            if score >= best_score:
                if saving:
                    print('saving model to... {}'.format(filepath))
                    save_model(model,filepath)
                    print('saving preds...')
                    pickle.dump(y_pred,open(predspath,'wb'))
        ROC_AUC_SCORE.append(score)
        if saving:
            print('saving report to... {}'.format(reportpath))
            pickle.dump(ROC_AUC_SCORE,open(reportpath,'wb'))
        print('======= current {} / {}'.format(epoch,total_epoch))
        print('previous best roc is {}'.format(best_score))
        print('current roc is {}'.format(score))
        try:
            best_round = ROC_AUC_SCORE.index(best_score)
        except ValueError:
            best_round = -1
        if len(ROC_AUC_SCORE) > patience + best_round:
            print('reach patience! end loop')
            break
            
def train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=512,total_epoch=400,patience=30):
    train_fold_index = pickle.load(open(ENV.train_fold_index.value,'rb'))
    val_fold_index = pickle.load(open(ENV.val_fold_index.value,'rb'))

    for fold in range(0,len(train_fold_index)):
        print('!!!!!!!! Begin fold: {}'.format(fold))
        train_index = train_fold_index[fold]
        val_index = val_fold_index[fold]
        X_tra = x_train[train_index]
        y_tra = y_train[train_index]
        X_val = x_train[val_index]
        y_val = y_train[val_index]
        print('preparing train/val done!')
        print('before evaluating: {}'.format(model_file))
        model_file_evl = model_file.format(fold)
        report_file_evl = report_file.format(fold)
        pred_file_evl = pred_file.format(fold)
        pred_test_file_evl = pred_test_file.format(fold)
        model = get_rnn_model(num_words,embed_size,embedding_matrix)
        train_each_fold(X_tra, y_tra, X_val, y_val,
                        model,
                        filepath=model_file_evl,reportpath=report_file_evl,predspath=pred_file_evl,
                        batch_size=batch_size,total_epoch=total_epoch,patience=patience)
        gc.collect()
        #### predict test
        model = load_model(model,model_file_evl)
        test_preds = model.predict(x_test,batch_size=1500,verbose=1)
        pickle.dump(test_preds,open(pred_test_file_evl,'wb'))
        print('\n')


# POS CASH V1

In [5]:
qt = QuantileTransformer(n_quantiles=10000,output_distribution='normal')
trans_col = ['MONTHS_BALANCE',
             'CNT_INSTALMENT',
             'NAME_CONTRACT_STATUS',
             'SK_DPD',
             'SK_DPD_DEF']
for col in trans_col:
    print(col)
    X_pos[col] = qt.fit_transform(X_pos[col].values.reshape(-1,1))

MONTHS_BALANCE
CNT_INSTALMENT
NAME_CONTRACT_STATUS
SK_DPD
SK_DPD_DEF


In [6]:

print('start')
label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)

#previous application
#10001358
max_features = 10001359
#295
maxlen = 50

sorted_df = X_pos.sort_values(['SK_ID_CURR','MONTHS_BALANCE'])
col = 'MONTHS_BALANCE'
sorted_df[col] = qt.fit_transform(sorted_df[col].values.reshape(-1,1))
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
feature.remove('SK_ID_PREV')
feature.remove('SK_ID_CURR')
feature.remove('words')
ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embed_size = len(feature)
print('ebd size is {}'.format(embed_size))
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
print('get embedding Mat')
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
model_file = ENV.POS_CASH_rnn.value
report_file = ENV.POS_CASH_report.value
pred_file = ENV.POS_CASH_preds.value
pred_test_file = ENV.POS_CASH_preds_test.value
train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=2000,total_epoch=500,patience=35)

start
start normalize
get embedding
ebd size is 6
create document
get embedding Mat
...get ebd mat
num of words: 10001359
(10001359, 6)
!!!!!!!! Begin fold: 0
preparing train/val done!
before evaluating: ../LIB/../../data/rnn/poscash/fold_{}.hdf5
Epoch 1/1
61503/61503 [==============================] - 0s 5us/step
saving preds...
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 0 / 500
previous best roc is 0
current roc is 0.5683583433524053
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving model to... ../LIB/../../data/rnn/poscash/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 1 / 500
previous best roc is 0.5683583433524053
current roc is 0.5830569282959639
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving model to... ../LIB/../../data/rnn/poscash/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/poscash/report_fold

61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 22 / 500
previous best roc is 0.6059031260939679
current roc is 0.6011215179539859
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 23 / 500
previous best roc is 0.6059031260939679
current roc is 0.6043172384950446
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 24 / 500
previous best roc is 0.6059031260939679
current roc is 0.6036546777349915
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 25 / 500
previous best roc is 0.6059031260939679
current roc is 0.6022340008091775
Epoch 1/1
61503/61503 [==============================] - 0s 3us/st

Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 47 / 500
previous best roc is 0.6063191262173531
current roc is 0.6049805901400853
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving model to... ../LIB/../../data/rnn/poscash/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 48 / 500
previous best roc is 0.6063191262173531
current roc is 0.6068478857366673
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 49 / 500
previous best roc is 0.6068478857366673
current roc is 0.6066705302265123
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 50 / 500
previous best roc is 0.6068478857366673
current roc i

Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 72 / 500
previous best roc is 0.6071376114771221
current roc is 0.606359545641058
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 73 / 500
previous best roc is 0.6071376114771221
current roc is 0.6063776714337363
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 74 / 500
previous best roc is 0.6071376114771221
current roc is 0.6052723011632646
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 75 / 500
previous best roc is 0.6071376114771221
current roc is 0.6063324349645052
Epoch 1/1
61503/61503 [==============================] - 

61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 97 / 500
previous best roc is 0.6071376114771221
current roc is 0.6052492363745401
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 98 / 500
previous best roc is 0.6071376114771221
current roc is 0.604398866434052
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 99 / 500
previous best roc is 0.6071376114771221
current roc is 0.6056591461455028
reach patience! end loop
48744/48744 [==============================] - 0s 5us/step


!!!!!!!! Begin fold: 1
preparing train/val done!
before evaluating: ../LIB/../../data/rnn/poscash/fold_{}.hdf5
Epoch 1/1
61502/61502 [==============================] - 0s 5us/step
saving preds...
saving report to... .

...

Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving model to... ../LIB/../../data/rnn/poscash/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 46 / 500
previous best roc is 0.6073200631998188
current roc is 0.6082989281530381
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving model to... ../LIB/../../data/rnn/poscash/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 47 / 500
previous best roc is 0.6082989281530381
current roc is 0.6086501800983062
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 48 / 500
previous best roc is 0.6086501800983062
current roc is 0.6058318740659163
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
=

61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 70 / 500
previous best roc is 0.6092047610248535
current roc is 0.60524658494827
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 71 / 500
previous best roc is 0.6092047610248535
current roc is 0.607167173186372
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 72 / 500
previous best roc is 0.6092047610248535
current roc is 0.6086745831924043
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 73 / 500
previous best roc is 0.6092047610248535
current roc is 0.6087549944251877
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step


Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 95 / 500
previous best roc is 0.6100402686492798
current roc is 0.6096624076396411
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 96 / 500
previous best roc is 0.6100402686492798
current roc is 0.6089232882589235
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 97 / 500
previous best roc is 0.6100402686492798
current roc is 0.6079061895012652
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 98 / 500
previous best roc is 0.6100402686492798
current roc is 0.6091673644042148
Epoch 1/1
61502/61502 [==============================] -

61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 119 / 500
previous best roc is 0.6107159428784177
current roc is 0.6076767179640824
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 120 / 500
previous best roc is 0.6107159428784177
current roc is 0.6079486508126694
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 121 / 500
previous best roc is 0.6107159428784177
current roc is 0.6086603384105945
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 122 / 500
previous best roc is 0.6107159428784177
current roc is 0.6097670447656318
Epoch 1/1
61502/61502 [==============================] - 0s 3u

61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 144 / 500
previous best roc is 0.610996846070056
current roc is 0.6070345365114697
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 145 / 500
previous best roc is 0.610996846070056
current roc is 0.6093935378350228
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 146 / 500
previous best roc is 0.610996846070056
current roc is 0.6085950163761107
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 147 / 500
previous best roc is 0.610996846070056
current roc is 0.6088052616166466
Epoch 1/1
61502/61502 [==============================] - 0s 3us/st

In [7]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()

count    307511.000000
mean         27.841092
std          24.514786
min           1.000000
25%          11.000000
50%          21.000000
75%          38.000000
max         295.000000
Name: text, dtype: float64

In [8]:
aaaa

NameError: name 'aaaa' is not defined

# POS CASH V2

In [4]:
qt = QuantileTransformer(n_quantiles=10000,output_distribution='normal')
trans_col = ['MONTHS_BALANCE_MAX',
             'MONTHS_BALANCE_MIN',
             'MONTHS_BALANCE_SPAN',
             'CNT_INSTALMENT_MAX',
             'CNT_INSTALMENT_MIN',
             'CNT_INSTALMENT_SPAN',
             'SK_DPD_MAX',
             'SK_DPD_MIN',
             'SK_DPD_MEAN',
             'SK_DPD_COUNT',
             'SK_DPD_SUM',
             'SK_DPD_DEF_MAX',
             'SK_DPD_DEF_MIN',
             'SK_DPD_DEF_MEAN',
             'SK_DPD_DEF_COUNT',
             'SK_DPD_DEF_SUM']
for col in trans_col:
    print(col)
    X_pos_fe[col] = qt.fit_transform(X_pos_fe[col].values.reshape(-1,1))

MONTHS_BALANCE_MAX
MONTHS_BALANCE_MIN
MONTHS_BALANCE_SPAN
CNT_INSTALMENT_MAX
CNT_INSTALMENT_MIN
CNT_INSTALMENT_SPAN
SK_DPD_MAX
SK_DPD_MIN
SK_DPD_MEAN
SK_DPD_COUNT
SK_DPD_SUM
SK_DPD_DEF_MAX
SK_DPD_DEF_MIN
SK_DPD_DEF_MEAN
SK_DPD_DEF_COUNT
SK_DPD_DEF_SUM


In [5]:

print('start')
label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)

#previous application
#10001358
max_features = 10001359
#295
maxlen = 12

sorted_df = X_pos_fe.sort_values(['SK_ID_CURR','MONTHS_BALANCE_MIN'])
col = 'MONTHS_BALANCE_MIN'
sorted_df[col] = qt.fit_transform(sorted_df[col].values.reshape(-1,1))
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
feature.remove('SK_ID_PREV')
feature.remove('SK_ID_CURR')
feature.remove('words')
ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embed_size = len(feature)
print('ebd size is {}'.format(embed_size))
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
print('get embedding Mat')
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
model_file = ENV.POS_CASH_rnn.value
report_file = ENV.POS_CASH_report.value
pred_file = ENV.POS_CASH_preds.value
pred_test_file = ENV.POS_CASH_preds_test.value
train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=2000,total_epoch=500,patience=35)

start
start normalize
get embedding
ebd size is 40
create document
get embedding Mat
...get ebd mat
num of words: 936327
(936327, 40)
!!!!!!!! Begin fold: 0
preparing train/val done!
before evaluating: ../LIB/../../data/rnn/poscash/fold_{}.hdf5
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving preds...
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 0 / 500
previous best roc is 0
current roc is 0.5783847115655003
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/poscash/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 1 / 500
previous best roc is 0.5783847115655003
current roc is 0.5915485061778698
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/poscash/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0

Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/poscash/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 22 / 500
previous best roc is 0.60866926158243
current roc is 0.6086898981211012
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/poscash/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 23 / 500
previous best roc is 0.6086898981211012
current roc is 0.6098793138202937
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 24 / 500
previous best roc is 0.6098793138202937
current roc is 0.6089516415974657
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
===

======= current 45 / 500
previous best roc is 0.6109310850439882
current roc is 0.6113821387682137
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 46 / 500
previous best roc is 0.6113821387682137
current roc is 0.6111018910938819
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 47 / 500
previous best roc is 0.6113821387682137
current roc is 0.6103020535032797
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/poscash/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 48 / 500
previous best roc is 0.6113821387682137
current roc is 0.6114095812219156
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/posca

Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 70 / 500
previous best roc is 0.6121746700879765
current roc is 0.6111703627238868
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 71 / 500
previous best roc is 0.6121746700879765
current roc is 0.6107218143654842
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 72 / 500
previous best roc is 0.6121746700879765
current roc is 0.6108655976149348
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 73 / 500
previous best roc is 0.6121746700879765
current roc is 0.6109165514114696
Epoch 1/1
61503/61503 [==============================] -

Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/poscash/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 95 / 500
previous best roc is 0.612326918138203
current roc is 0.6126650958961498
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/poscash/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 96 / 500
previous best roc is 0.6126650958961498
current roc is 0.6133165788862044
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
======= current 97 / 500
previous best roc is 0.6133165788862044
current roc is 0.6124720553970996
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_0.pkl
==

...

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_3.pkl
======= current 172 / 500
previous best roc is 0.6074695087206913
current roc is 0.6062066149097832
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_3.pkl
======= current 173 / 500
previous best roc is 0.6074695087206913
current roc is 0.6054445833644646
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_3.pkl
======= current 174 / 500
previous best roc is 0.6074695087206913
current roc is 0.6056039770336328
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_3.pkl
======= current 175 / 500
previous best roc is 0.6074695087206913
current roc is 0.6061007284554771
Epoch 1/1
61502/61502 [=============================

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 12 / 500
previous best roc is 0.6005798385629004
current roc is 0.6004576386588292
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 13 / 500
previous best roc is 0.6005798385629004
current roc is 0.6005673820635493
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/poscash/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 14 / 500
previous best roc is 0.6005798385629004
current roc is 0.6024820159900875
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 15 / 500
previous best roc is 0.6024820159900875
current roc i

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 36 / 500
previous best roc is 0.6062985958977516
current roc is 0.6050785225605954
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 37 / 500
previous best roc is 0.6062985958977516
current roc is 0.605434463096779
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 38 / 500
previous best roc is 0.6062985958977516
current roc is 0.6056326333623306
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 39 / 500
previous best roc is 0.6062985958977516
current roc is 0.6051802539536122
Epoch 1/1
61502/61502 [==============================] - 

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 61 / 500
previous best roc is 0.6078467476432787
current roc is 0.6067954491137048
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 62 / 500
previous best roc is 0.6078467476432787
current roc is 0.6066702277536288
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 63 / 500
previous best roc is 0.6078467476432787
current roc is 0.6053699890951294
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 64 / 500
previous best roc is 0.6078467476432787
current roc is 0.606098402287335
Epoch 1/1
61502/61502 [==============================] - 

61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 86 / 500
previous best roc is 0.6078467476432787
current roc is 0.6055643513497022
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 87 / 500
previous best roc is 0.6078467476432787
current roc is 0.6070815544172073
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 88 / 500
previous best roc is 0.6078467476432787
current roc is 0.6075416402565013
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/poscash/report_fold_4.pkl
======= current 89 / 500
previous best roc is 0.6078467476432787
current roc is 0.6050858040706675
Epoch 1/1
61502/61502 [==============================] - 0s 2us/st

In [10]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()

count    307511.000000
mean          2.661381
std           1.949228
min           1.000000
25%           1.000000
50%           2.000000
75%           3.000000
max          26.000000
Name: text, dtype: float64

In [ ]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()